In [ ]:
import os
import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Загружаем модель SpaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

In [ ]:
# Пример классификатора (слова, их категории, определения и группы классификации)
classifier = {
    "Документ": {"category": "Классификатор документа", "definition": "Официальный текст, содержащий информацию", "group": "Официальные документы"},
    "Требования": {"category": "Классификатор требования", "definition": "Условия, которым должен соответствовать объект", "group": "Технические требования"},
    "Безопасность": {"category": "Классификатор безопасности", "definition": "Состояние защищенности от угроз", "group": "Нормативные акты"},
    "Строительство": {"category": "Классификатор строительства", "definition": "Процесс возведения зданий и сооружений", "group": "Строительные нормы"}
}

In [ ]:

# Исходный текст
text = "Этот документ описывает требования к безопасности строительства."

In [ ]:

# Функция предобработки текста с использованием SpaCy
def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha]
    return tokens

# Разбиваем текст на слова
words = preprocess_text(text)

# Векторизация: ключевые слова классификатора + слова из текста
vectorizer = TfidfVectorizer()
all_words = list(classifier.keys()) + words
tfidf_matrix = vectorizer.fit_transform(all_words)

# Разделяем векторы
word_vectors = tfidf_matrix[len(classifier):]  # Вектора слов из текста
class_vectors = tfidf_matrix[:len(classifier)]  # Вектора понятий из классификатора

# Вычисляем схожесть
cosine_similarities = cosine_similarity(word_vectors, class_vectors)

# Создаем итоговый список с классификацией каждого слова
classified_words = []
for i, word in enumerate(words):
    max_index = np.argmax(cosine_similarities[i])
    max_similarity = cosine_similarities[i, max_index]
    
    # Если схожесть ниже 0.2 – считаем слово неклассифицированным
    if max_similarity >= 0.2:
        category_data = classifier[list(classifier.keys())[max_index]]
        category = {"category": category_data["category"], "definition": category_data["definition"], "group": category_data["group"]}
    else:
        category = "не классифицировано"
    
    classified_words.append({word: category})

# Итоговый результат
result = {text: classified_words}





In [ ]:
# Вывод результата в виде таблицы
df_result = pd.DataFrame(classified_words)
print(df_result)


In [ ]:
# Также печатаем JSON-результат
import json
print(json.dumps(result, ensure_ascii=False, indent=4))